# NFT Market Analysis Using Covalent API
## Comprehensive Cross-Chain NFT Data Analysis

**Author:** Jakob Richert  
**Gitcoin Bounty:** [Covalent NFT CryptoSheets Challenge](https://gitcoin.co/issue/covalenthq/covalent-gitcoin-bounties/18/100027635)

### Project Overview
This notebook provides a comprehensive analysis of NFT market data across multiple blockchain networks using the Covalent API. The analysis covers:
- **Ethereum** (Chain ID: 1)
- **Polygon** (Chain ID: 137)
- **Avalanche** (Chain ID: 43114)
- **Fantom** (Chain ID: 250)

### Features
✓ Real-time NFT market data aggregation  
✓ SQL-based querying and analysis  
✓ Interactive data visualizations  
✓ Cross-chain comparative analytics  

## 1. Environment Setup & Dependencies

In [ ]:
# Import required libraries
import requests
import json
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print('✓ All dependencies loaded successfully')

## 2. Data Collection from Covalent API

Fetching NFT market data from multiple blockchain networks using Covalent's powerful API.

In [ ]:
# API Configuration
API_KEY = 'ckey_762a27bc3b6f4f2dbc03aeb8554'
BASE_URL = 'https://api.covalenthq.com/v1'

# Blockchain configurations
CHAINS = {
    'Ethereum': {'id': 1, 'page_size': 10000},
    'Polygon': {'id': 137, 'page_size': 1000000},
    'Avalanche': {'id': 43114, 'page_size': 1000000},
    'Fantom': {'id': 250, 'page_size': 1000000}
}

def fetch_nft_data(chain_id, page_size):
    """Fetch NFT market data for a specific blockchain."""
    url = f'{BASE_URL}/{chain_id}/nft_market/?format=JSON&page-size={page_size}&key={API_KEY}'
    try:
        response = requests.get(url)
        data = json.loads(response.text)
        return data['data']['items']
    except Exception as e:
        print(f'Error fetching data: {e}')
        return []

# Collect data from all chains
print('Fetching NFT market data from multiple blockchains...')
all_nft_data = []

for chain_name, config in CHAINS.items():
    print(f'  → Fetching {chain_name} data...')
    chain_data = fetch_nft_data(config['id'], config['page_size'])
    all_nft_data.extend(chain_data)
    print(f'    ✓ Retrieved {len(chain_data)} collections')

print(f'\n✓ Total collections retrieved: {len(all_nft_data)}')

## 3. Data Processing & Storage

In [ ]:
# Convert to DataFrame
df = pd.DataFrame(all_nft_data)

# Save to Excel
df.to_excel('nft_market_data.xlsx', index=False)
print('✓ Data saved to nft_market_data.xlsx')

# Create SQLite database
conn = sqlite3.connect('nft_market_data.db')
df.to_sql('nft_collections', conn, if_exists='replace', index=False)
print('✓ SQLite database created: nft_market_data.db')

# Display data summary
print(f'\nDataset Shape: {df.shape[0]} rows × {df.shape[1]} columns')
print(f'\nColumn Names:')
for col in df.columns:
    print(f'  • {col}')

## 4. Data Preview

In [ ]:
# Display first few rows
print('First 5 Collections:')
df.head()

In [ ]:
# Display last few rows
print('Last 5 Collections:')
df.tail()

## 5. SQL Query Analysis

Using SQL queries to extract insights from the NFT market data.

In [ ]:
cursor = conn.cursor()

# Define all queries
queries = {
    'query1': {
        'sql': '''SELECT collection_name, market_cap_quote 
                  FROM nft_collections 
                  WHERE chain_id = 1 
                  ORDER BY market_cap_quote DESC 
                  LIMIT 5''',
        'description': 'Top 5 Ethereum Collections by Market Cap'
    },
    'query2': {
        'sql': '''SELECT collection_name, market_cap_quote 
                  FROM nft_collections 
                  WHERE chain_id = 137 
                  ORDER BY market_cap_quote DESC 
                  LIMIT 5''',
        'description': 'Top 5 Polygon Collections by Market Cap'
    },
    'query3': {
        'sql': '''SELECT collection_name, market_cap_quote 
                  FROM nft_collections 
                  WHERE chain_id = 43114 
                  ORDER BY market_cap_quote DESC 
                  LIMIT 5''',
        'description': 'Top 5 Avalanche Collections by Market Cap'
    },
    'query4': {
        'sql': '''SELECT collection_name, market_cap_quote 
                  FROM nft_collections 
                  WHERE chain_id = 250 
                  ORDER BY market_cap_quote DESC 
                  LIMIT 5''',
        'description': 'Top 5 Fantom Collections by Market Cap'
    },
    'query5': {
        'sql': '''SELECT COUNT(*) as count, chain_id 
                  FROM nft_collections 
                  GROUP BY chain_id 
                  ORDER BY count DESC''',
        'description': 'NFT Collections Count by Blockchain'
    },
    'query6': {
        'sql': '''SELECT collection_name, contract_deployment_at 
                  FROM nft_collections 
                  WHERE chain_id = 1 AND contract_deployment_at IS NOT NULL 
                  ORDER BY contract_deployment_at 
                  LIMIT 1''',
        'description': 'Earliest Ethereum NFT Contract Deployment'
    },
    'query7': {
        'sql': '''SELECT collection_name, contract_deployment_at 
                  FROM nft_collections 
                  WHERE chain_id = 137 AND contract_deployment_at IS NOT NULL 
                  ORDER BY contract_deployment_at 
                  LIMIT 1''',
        'description': 'Earliest Polygon NFT Contract Deployment'
    },
    'query8': {
        'sql': '''SELECT collection_name, contract_deployment_at 
                  FROM nft_collections 
                  WHERE chain_id = 43114 AND contract_deployment_at IS NOT NULL 
                  ORDER BY contract_deployment_at 
                  LIMIT 1''',
        'description': 'Earliest Avalanche NFT Contract Deployment'
    },
    'query9': {
        'sql': '''SELECT collection_name, contract_deployment_at 
                  FROM nft_collections 
                  WHERE chain_id = 250 AND contract_deployment_at IS NOT NULL 
                  ORDER BY contract_deployment_at 
                  LIMIT 1''',
        'description': 'Earliest Fantom NFT Contract Deployment'
    },
    'query10': {
        'sql': '''SELECT collection_name, transaction_count_alltime, chain_id 
                  FROM nft_collections 
                  ORDER BY transaction_count_alltime DESC 
                  LIMIT 1''',
        'description': 'Highest Transaction Volume NFT Collection (All Chains)'
    }
}

# Execute and display results
results = {}
for query_name, query_info in queries.items():
    print(f"\n{'='*60}")
    print(f"{query_info['description']}")
    print('='*60)
    cursor.execute(query_info['sql'])
    result = cursor.fetchall()
    results[query_name] = result
    for row in result:
        print(row)

## 6. Data Visualizations

### 6.1 NFT Collections Distribution by Blockchain

In [ ]:
# Chain distribution
chain_counts = pd.DataFrame(results['query5'], columns=['Count', 'Chain_ID'])
chain_names = {1: 'Ethereum', 137: 'Polygon', 43114: 'Avalanche', 250: 'Fantom'}
chain_counts['Blockchain'] = chain_counts['Chain_ID'].map(chain_names)

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Bar chart
colors = ['#627EEA', '#8247E5', '#E84142', '#1969FF']
ax1.bar(chain_counts['Blockchain'], chain_counts['Count'], color=colors)
ax1.set_title('NFT Collections Count by Blockchain', fontsize=14, fontweight='bold')
ax1.set_ylabel('Number of Collections', fontsize=12)
ax1.set_xlabel('Blockchain', fontsize=12)
for i, v in enumerate(chain_counts['Count']):
    ax1.text(i, v + 100, str(v), ha='center', va='bottom', fontweight='bold')

# Pie chart
ax2.pie(chain_counts['Count'], labels=chain_counts['Blockchain'], autopct='%1.1f%%', 
        colors=colors, startangle=90)
ax2.set_title('Market Share by Blockchain', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### 6.2 Top Collections by Market Cap (Per Chain)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

queries_to_plot = ['query1', 'query2', 'query3', 'query4']
chain_names_ordered = ['Ethereum', 'Polygon', 'Avalanche', 'Fantom']
colors_ordered = ['#627EEA', '#8247E5', '#E84142', '#1969FF']

for idx, (query_key, chain_name, color) in enumerate(zip(queries_to_plot, chain_names_ordered, colors_ordered)):
    data = pd.DataFrame(results[query_key], columns=['Collection', 'Market_Cap'])
    
    axes[idx].barh(data['Collection'], data['Market_Cap'], color=color)
    axes[idx].set_title(f'Top 5 {chain_name} Collections', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Market Cap (USD)', fontsize=10)
    axes[idx].invert_yaxis()
    
    # Add value labels
    for i, v in enumerate(data['Market_Cap']):
        if v and pd.notna(v):
            axes[idx].text(v, i, f' ${v:,.0f}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

### 6.3 Historical NFT Contract Deployments Timeline

In [ ]:
# Extract deployment dates
deployment_data = []
for query_key, chain_name in zip(['query6', 'query7', 'query8', 'query9'], 
                                   ['Ethereum', 'Polygon', 'Avalanche', 'Fantom']):
    result = results[query_key][0]
    deployment_data.append({
        'Blockchain': chain_name,
        'Collection': result[0],
        'Date': result[1],
        'Year': result[1][:4] if result[1] else 'N/A'
    })

deployment_df = pd.DataFrame(deployment_data)

print("\n" + "="*70)
print("EARLIEST NFT CONTRACT DEPLOYMENTS BY BLOCKCHAIN")
print("="*70)
for _, row in deployment_df.iterrows():
    print(f"\n{row['Blockchain']:12} | {row['Collection']:30} | {row['Date']}")
print("="*70)

## 7. Key Insights & Summary Statistics

In [ ]:
# Most active collection
most_active = results['query10'][0]
most_active_chain = {1: 'Ethereum', 137: 'Polygon', 43114: 'Avalanche', 250: 'Fantom'}[most_active[2]]

print("\n" + "="*70)
print("KEY INSIGHTS")
print("="*70)
print(f"\n📊 Total NFT Collections Analyzed: {len(df):,}")
print(f"\n🏆 Most Active Collection (All Chains):")
print(f"   • Collection: {most_active[0]}")
print(f"   • Blockchain: {most_active_chain}")
print(f"   • All-Time Transactions: {most_active[1]:,}")
print(f"\n🔗 Blockchain Distribution:")
for _, row in chain_counts.iterrows():
    pct = (row['Count'] / len(df)) * 100
    print(f"   • {row['Blockchain']:12} → {row['Count']:5,} collections ({pct:.1f}%)")
print("\n" + "="*70)

## 8. Additional Analysis - Market Cap Statistics

In [ ]:
# Market cap analysis by chain
market_cap_query = '''
    SELECT 
        chain_id,
        COUNT(*) as collection_count,
        AVG(market_cap_quote) as avg_market_cap,
        MAX(market_cap_quote) as max_market_cap,
        SUM(market_cap_quote) as total_market_cap
    FROM nft_collections
    WHERE market_cap_quote IS NOT NULL
    GROUP BY chain_id
    ORDER BY total_market_cap DESC
'''

cursor.execute(market_cap_query)
market_stats = cursor.fetchall()

market_df = pd.DataFrame(market_stats, 
                         columns=['Chain_ID', 'Collections', 'Avg_Market_Cap', 'Max_Market_Cap', 'Total_Market_Cap'])
market_df['Blockchain'] = market_df['Chain_ID'].map(chain_names)

print("\n" + "="*90)
print("MARKET CAP STATISTICS BY BLOCKCHAIN")
print("="*90)
print(f"{'Blockchain':<12} {'Collections':>12} {'Avg Market Cap':>20} {'Total Market Cap':>25}")
print("-"*90)
for _, row in market_df.iterrows():
    print(f"{row['Blockchain']:<12} {row['Collections']:>12,} {f'${row["Avg_Market_Cap"]:,.2f}':>20} {f'${row["Total_Market_Cap"]:,.2f}':>25}")
print("="*90)

## 9. Conclusion

This analysis provides comprehensive insights into the NFT market across multiple blockchain networks. Key findings:

1. **Ethereum dominates** the NFT market in terms of collection count
2. **Cross-chain adoption** is growing, with Polygon showing significant activity
3. **Market concentration** varies significantly across different blockchains
4. **Historical perspective** shows Ethereum's early NFT adoption (2017)

### Data Availability
- **Excel Export:** `nft_market_data.xlsx`
- **SQLite Database:** `nft_market_data.db`
- **Source Code:** Available on GitHub

### License
MIT License - Free to use for analysis and research

In [ ]:
# Close database connection
conn.close()
print('\n✓ Analysis complete! Database connection closed.')